# Pandas tutorial

Dataset characteristic:

- every folder contain: addresses.csv, addresses_people.csv, people.csv, people_publications.csv, publications.csv.
- every file contain few columns, named in first row
- every file not with many-to-many relations, contain temp_id column

Project:

- merge all files into single DataFrame
- change current temp_ids into new unique ids
- eliminate duplicates (eg. consider address with x percentage of similarity as one)
- add column “town” for people.csv data, extracted from “addresses” column in addresses.csv
- get missing lat/lng data for every town/address (eg. through google api)
- save DataFrame to single csv

## Variables

In [10]:
similarity_percentage = 20

data_directory="./data"
data_sub_folders = 134
data_load_step = 1

## Functions & Imports

In [45]:
import re
import pandas as pd
import os

def move_head(arr, index: int):
    arr = arr = [arr[index]] + arr[:index] + arr[index + 1:]

def data_filter(name: str, address: str):
    data = name.split(",") + address.split(",")
    filtered_data = []
    for word in data:
        cleaned_word = word.strip().lower()
        cleaned_word = re.sub(r'[^a-zA-Z\s]', ' ', cleaned_word)
        cleaned_word = re.sub(r'\s+', ' ', cleaned_word)
        if len(cleaned_word) > 2:
            filtered_data.append(cleaned_word.strip())
    return ",".join(filtered_data)
            
def similarity(data1: str, data2: str):
    arr1, arr2 = data1.split(","), data2.split(",")
    counter = 0
    total = max(len(arr1), len(arr2))
    for str1 in arr1:
        for str2 in arr2:
            if str1 in str2 or str2 in str1:
                counter += 1
    return counter / total * 100

def make_id_unique(df, value, column="temp_id"):
    df[column] = df[column].apply(lambda id: str(value) + "_" + str(id))
    return df

## Load data

In [49]:
data_files = {"addresses": [], "addresses_people": [], "people": [], "people_publications": [], "publications": []}

for i in range(0, data_sub_folders, data_load_step):
    data_files["addresses"].append(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/ADDRESSES.csv")), i))
    data_files["addresses_people"].append(make_id_unique(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/ADDRESSES_PEOPLE.csv")), i, "address_uuid"), i, "person_uuid"))
    data_files["people"].append(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/PEOPLE.csv")), i))
    data_files["people_publications"].append(make_id_unique(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/PEOPLE_PUBLICATIONS.csv")), i, "person_uuid"), i, "publication_uuid"))
    data_files["publications"].append(make_id_unique(pd.read_csv(os.path.join(data_directory, f"{i}/PUBLICATIONS.csv")), i))

address_df = pd.concat(data_files["addresses"], ignore_index=True)
addresses_people_df = pd.concat(data_files["addresses_people"], ignore_index=True)
people_df = pd.concat(data_files["people"], ignore_index=True)
people_publications_df = pd.concat(data_files["people_publications"], ignore_index=True)
publications_df = pd.concat(data_files["publications"], ignore_index=True)

addresses_people_df.head()

,address_uuid,person_uuid
0,0_2,0_1
1,0_3,0_2
2,0_4,0_2
3,0_4,0_3
4,0_5,0_4


## Prepare good addresses indexes

In [51]:
%%time

address_df["full_address"] = address_df.apply(lambda x: data_filter(str(x["name"]), str(x["address"])), axis=1)

good_address_list = []  # (address["full_address"], address["temp_uuid"], [])

for address_index, address in address_df.iterrows():
    flag = True
    for good_address_index, ga in enumerate(good_address_list):
        if similarity(ga[0], address["full_address"]) > similarity_percentage:
            flag = False
            ga[2].append(address["temp_uuid"])
            move_head(good_address_list, good_address_index)
            break
    if flag:    
        good_address_list.append((address["full_address"], address["temp_uuid"], []))

address_df = address_df.drop(["full_address"], axis=1)

print(f"{len(good_address_list)} good addresses found")

KeyError: 'temp_uuid'

## Update ADDRESSES_PEOPLE relation 

In [37]:
for address_str, good_address_index, to_update_address_list in good_address_list:
    for to_replace_address_index in to_update_address_list:
        addresses_people_df.loc[to_replace_address_index, ""]

0
1
20
30
44
59
65
82
118
224
352
360
457
462
492
552
554
715
735
905
937
959
1033
1224
1227
1467
1646
1875
1979
2068
2197
2198
2250
2305
2319
2944
3165
3289
3532
3659
4402
4480
6220
6511
7163
7280
7608
7953
8251
8314
8460
8773
9364
9590
10151
10394
10695
11112
11986
12122
12426
12656
14007
14475
15182
15286
15500
16193
16655
17594
17856
18094
18271
18470
19612
20534
20680
20726
21031
21071
21681
21709
22341
24264
26195
26644
26916
28088
28118
28263
28754
29347
29838
29846
30297
32032
33644
33940
34625
34951
35697
36223
39539
40279
40692
41002
42307
43257
43361
43744
43783
44377
44614
46155
46437
47242
47347
47931
48809
49394
49909
50234
51664
52055
52145
52391
55435
56134
57189
57628
58375
59828
61249
65275
66080
66915
68228
68767
69240
70933
73047
76726
76997
77175
78458
78595
80038
81685
82785
83886
85936
86779
87078
88047
91325
91675
91716
92126
92529
94207
98150
99486
101243
101363
102254
103766
105767
108252
109258
109412
110966
111738
117178
117537
117936
119060
119680
119877
12

# Merge

In [ ]:
import os
import pandas as pd

dfs = []

for it in os.scandir("./data"):
    print(it)

for dirpath, dirnames, filenames in os.walk("./data/"):
    if dirnames:
        continue
    addresses = pd.read_table(os.path.join(dirpath, "ADDRESSES.csv"), sep=',')
    addresses_people = pd.read_table(os.path.join(dirpath, "ADDRESSES_PEOPLE.csv"), sep=',')
    people = pd.read_table(os.path.join(dirpath, "PEOPLE.csv"), sep=",")
    people_publications = pd.read_table(os.path.join(dirpath, "PEOPLE_PUBLICATIONS.csv"), sep=",")
    publications = pd.read_table(os.path.join(dirpath, "PUBLICATIONS.csv"), sep=",")

    addresses.rename(columns={"temp_id": "address_uuid"}, inplace=True)
    people.rename(columns={"temp_id": "person_uuid"}, inplace=True)
    publications.rename(columns={"temp_id": "publication_uuid"}, inplace=True)
    
    merged_1 = pd.merge(addresses, addresses_people, on='address_uuid', how="inner")
    merged_2 = pd.merge(merged_1, people, on="person_uuid", how="inner")
    merged_3 = pd.merge(merged_2, people_publications, on="person_uuid", how="inner")
    data = pd.merge(merged_3, publications, on="publication_uuid", how="inner")

    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df.head()

In [ ]:
df['id'] = df['address_uuid'].astype(str) + "_" + df['person_uuid'].astype(str) + "_" + df['publication_uuid'].astype(str)
df = df[['id'] + [col for col in df.columns if col != 'id']]
df.drop(columns=['address_uuid', 'person_uuid', "publication_uuid"], inplace=True)

In [ ]:
df.groupby('address').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [ ]:
visited = set()
c = 0
i = 0
for index, row in df.iterrows():
    i+=1
    current_address = row['address']
    current_name = row['name']
    v = str(current_address) + ", " + str(current_name)
    if v in visited:
        c += 1
        continue
    visited.add(v)

c, i, c-i
    
    

In [ ]:
df.at[1, "firstname"] = "Paweł"
df.iloc[1]